In [3]:
# Data Manipulation and Math
import pandas as pd
import numpy as np
import openpyxl as xl 
import scipy as sp

# Graphing Libraries
import plotly.express as px
import plotly.graph_objects as go

# 
from dataclasses import dataclass
import calendar



In [5]:
# Ingests xlsx or csv. Try-except version
# Expects a column format of datetime, wind speed, gust speed, wind direction in degrees
def data_ingest(file : str = None) -> pd.DataFrame:
    
    if file is None:
        raise ValueError("No file provided")
    
    try:
        dataframe = pd.read_excel(file,names=["Date-Time","Wind-Speed","Gust-Speed","Wind-Direction"])
    except Exception as excel_error:
        try:
            dataframe = pd.read_csv(file,names=["Date-Time","Wind-Speed","Gust-Speed","Wind-Direction"])
        except Exception as csv_error:
            raise ValueError(f"Failed to read file. Excel Error: {excel_error}; CSV Error {csv_error}")
    
    return dataframe

In [6]:
# removes null values. Not useful for wind data assessment.
def data_clean(dataframe : pd.DataFrame = None) -> pd.DataFrame:
    if dataframe is None:
        raise ValueError("No dataframe given")
    
    dataframe = dataframe.dropna().reset_index(drop=True)
    
    dataframe["Date-Time"] = pd.to_datetime(dataframe["Date-Time"])
    
    return dataframe

In [10]:
# aggregates data to 
def data_aggregate(dataframe : pd.DataFrame = None) -> pd.DataFrame:
    if dataframe is None:
        raise ValueError("No dataframe given")
    
    processing = dataframe.drop(["Wind-Speed","Gust-Speed","Wind-Direction"],axis=1)
    
    processing = processing.groupby(["Year","Month"])["Day"].nunique().reset_index(name="Days-Counted")  
    
    processing = pd.concat([processing,pd.DataFrame([{"Year": 2010, "Month" : 2, "Days-Counted" :0}])],ignore_index = True)
    processing = processing.sort_values(by=["Year","Month"]).reset_index(drop=True)
    
    processing["Year-Month"] = processing.apply(func= lambda row: f"{row["Year"]}-{calendar.month_name[row["Month"]]}",axis=1)
    
    return processing


In [21]:
def plot_availability(dataframe : pd.DataFrame = None) -> None:
    
    fig = px.bar(
        data_frame=dataframe,
        y="Year-Month",
        x="Days-Counted",
        color="Days-Counted",
        text="Days-Counted",
        width=650,
        color_continuous_scale="portland"
        
    )
    
    fig.update_traces(
        textposition="outside"
    )
    
    fig.show()

In [22]:
plot_availability(agg)

# Diurnal Variation

The diurnal variation of wind speed provides information about the availability of suitable winds during the 24 h of the day

This pattern requires the overall hoyrly mean values of wind speed

It looks like a dot plot with a line 

In [14]:
def diurnal_variation(dataframe : pd.DataFrame = None, month : int = None, day : int = None) -> pd.DataFrame:
    if dataframe is None:
        raise ValueError("No dataframe given")
    
    if None in (month,day):
        raise ValueError("No month/day given")
    
    if not 1 <= month <= 12:
        raise ValueError("Invalid month")
    
    if not 1 <= day <= 31:
        raise ValueError("Invalid day")
    
    processing = dataframe.drop(["Date-Time","Gust-Speed","Wind-Direction","Year"], axis=1)
    
    filtering = processing[(processing['Month'] == month) & (processing['Day'] == day)]
    
    # Using the aggregate method instead of directly chaining .mean() returns a dataframe by default
    # You can also chain together dot notation when inside a parenthesis to indicate line continuation
    diurnal = (filtering.groupby("Hour")
               .aggregate({"Wind-Speed" : "mean"})
               .rename(columns={"Wind-Speed":"Average-Wind-Speed"})
               .reset_index())
    
    return diurnal

In [15]:
diurnal = diurnal_variation(dates,7,1)

diurnal

,Hour,Average-Wind-Speed
0,1,0.928333
1,2,0.542500
2,3,0.773333
3,4,1.035000
4,5,1.035833
5,6,1.050000
6,7,0.927500
7,8,1.035000
8,9,1.407500
9,10,1.330000



Continuous Color Scales
- plasma (default)
- viridis
- cividis
- portland
- tealgrn
- sunsetdark
- bluegrn
- darkmint
- 

In [16]:
def plot_diurnal(dataframe : pd.DataFrame = None) -> None:
    if dataframe is None:
        raise ValueError("No dataframe given")
    
    fig = px.scatter(
        data_frame=dataframe,
        x="Hour",  
        y="Average-Wind-Speed",
        color="Average-Wind-Speed",
        color_continuous_scale="tealgrn",
        range_x=[0,25],
        trendline="lowess",
        trendline_options=dict(frac=0.15)
    )   
    
    fig.update_traces(marker_size=15)
    
    
    fig.show()
    

In [17]:
plot_diurnal(diurnal)

# Wind Rose Diagram



In [18]:
def wind_rose(dataframe : pd.DataFrame = None, month : int = None, day : int = None) -> pd.DataFrame:
    

SyntaxError: expected ':' (2616722860.py, line 1)

In [ ]:
def plot_wind_rose(dataframe : pd.DataFrame = None) -> None:
    if dataframe is None:
        raise ValueError("No dataframe given")
    
    
    fig = px.bar_polar(
        data_frame=dataframe,
        r="Wind-Speed",
        theta="Wind-Direction",
        template="plotly"
    )
    
    fig.show()

In [ ]:
plot_wind_rose(dates)

# Frequency Distribution
Wind speed is not constant with time. Another way of assessing the availability of your wind data is the frequency distribution. 

In [ ]:
def frequency_distribution():
    

SyntaxError: incomplete input (424930223.py, line 2)

In [ ]:
def plot_frequency_distribution() -> None:
    

# Wind Speed Distribution and Weibull function


# Periodic Energy Production YEY (vm)


In [ ]:
powerCurve_df = pd.DataFrame({"Wind Speed": [speed * 0.5 for speed in range(51)],
                              "Power" : [0,0,0,0,0,0,0,6,13,47,81,125,169,229,
                                         289,369,448,546,644,751,858,964,1069,
                                         1166,1263,1347,1431,1492,1552,1585,1617,
                                         1631,1642,1647,1649,1650,1650,1650,1650,1650,
                                         1650,1650,1650,1650,1650,1650,1650,1650,1650,
                                         1650,1650]})


In [ ]:
class WindTurbine:
    def __init__(self, model : str = "V66/1650", ratedPower : int = 66,
                rotorDiameter : int = 66, numberBlades : int = 3,
                cutInSpeed : float = 3.5, ratedSpeed : float = 17.5,
                cutOffSpeed : int = 25, minHubHeight : int = 60,
                maxHubHeight : int = 78, powerCurve : pd.DataFrame = None):
        
        # No @property and @property.setter for now 
        
        self.model = model
        self.ratedPower = ratedPower
        self.rotorDiameter = rotorDiameter
        self.numberBlades = numberBlades
        self.cutInSpeed = cutInSpeed
        self.ratedSpeed = ratedSpeed
        self.cutOffSpeed = cutOffSpeed
        self.minHubHeight = minHubHeight
        self.maxHubHeight = maxHubHeight
        self.powerCurve = powerCurve if powerCurve is not None else self.defaultPowerCurve()
        
    def defaultPowerCurve(self):
        return pd.DataFrame({"Wind Speed": [speed * 0.5 for speed in range(51)],
                             "Power": [0, 0, 0, 0, 0, 0, 0, 6, 13, 47, 81, 125, 169, 229,
                                       289, 369, 448, 546, 644, 751, 858, 964, 1069,
                                       1166, 1263, 1347, 1431, 1492, 1552, 1585, 1617,
                                       1631, 1642, 1647, 1649, 1650, 1650, 1650, 1650, 1650,
                                       1650, 1650, 1650, 1650, 1650, 1650, 1650, 1650, 1650,
                                       1650, 1650]})
    
    def getYEY(self, distributionFunction : pd.DataFrame = None, hours : int = 8760):
        ... 
        # YEY = 0 
        # for i in range(26):
        #     YEY += distributionFunction of i  * self.powerCurve of i * hours
            
        

# Power Curve

